In [1]:
#RF12A

import numpy as np
import pandas as pd
from collections import namedtuple
from math import log10
from copy import copy, deepcopy
from sklearn.neighbors import KDTree
from collections import Counter
import os
import sys
np.set_printoptions(threshold=sys.maxsize)


Mol = namedtuple('Mol', ['symbols', 'coords', 'message'])

#symbols is a list of atomic name to be met in the pdb
#kdtrees - is a dictionary of the KD-trees for each element

PDB = namedtuple('PDB', ['symbols', 'kdtrees', 'coords'])


def read_xyz(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    message = data[1].rstrip("\n")
    symbols = []
    coords = np.array([]).reshape((0, 3)).astype(float)
    for item in data[2:]:
        if item[0] != "H":
            item = item.strip("\n")
            item = item.split(" ")
            item = list(filter(None, item))
            symbols.append(item[0])
            coords = np.concatenate((coords, np.array(item[1:]).reshape((1,3)).astype(float)), 0)
    mol = Mol(symbols = symbols, coords = coords, message = message)
    mol.coords.astype(float)
    return mol


def read_pdb(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    # retain only lines containing atomic coordinates
    data = [line.rstrip("\n") for line in data if line.startswith("ATOM") or line.startswith("HETATM")]
    symbols = ['C', 'O', 'N', 'S', 'P', 'M1', 'M2']
    coords = {'C': [], 'O': [], 'N': [], 'S': [], 'P': [], 'M1': [], 'M2': []}
    # read and keep coordinate per atom type
    for line in data:
        x = float(line[30:37])
        y = float(line[38:45])
        z = float(line[46:53])

        if len(line) < 80 or line[76:78] == "  ":
            if line.startswith("ATOM") or line[17:20] in ["CSO", "IAS", "PTR", "TPO", "SEP", "ACE", "PCA", "CSD", "LLP", "KCX"]:
                symbol = deepcopy(line[12:16]).strip()[0]
            elif line[17:20] == "MSE":
                symbol = deepcopy(line[12:16]).strip()
                if symbol != "SE":
                    symbol = symbol[0]
            else:
                symbol = deepcopy(line[12:16]).strip()

        elif line[76] == " ":
            symbol = line[77]

        else:
            symbol = line[76:78]

        if symbol in ['C', 'C1+', 'O', 'O1-', 'N', 'N1+', 'S', 'P', 'Cl']:
            
            if len(symbol) != 1:
                symbol = symbol[0]
            coords[symbol].append([x, y, z])
        elif symbol in ['LI', 'NA', 'K', 'RB', 'CS']:
            coords['M1'].append([x, y, z])
        elif symbol in ['AU', 'BA', 'CA', 'CD', 'SR', 'BE', 'CO', 'MG', 'CU', 'FE', 'HG', 'MN', 'NI', 'ZN' , 'Zn2+' , 'Zn']:
            coords['M2'].append([x, y, z])
        elif symbol == 'SE':
            coords['S'].append([x, y, z])
        elif symbol == 'H':
            continue
        else:
            print(symbol, line[0:78])
            raise ValueError("Unknown atomic symbol in pdb")

    kdtrees = {}

    # create a dict of KD trees
    for atom_type in symbols:
        coords[atom_type] = np.array(coords[atom_type])
        if len(coords[atom_type]) != 0:
            kdtrees[atom_type] = KDTree(coords[atom_type], leaf_size=10)

    pdb = PDB(symbols=symbols, kdtrees=kdtrees, coords=coords)

    return pdb

 
     

def RF_desc(mol, pdb):

    desc = [0.] * 63
    for atom_type in pdb.coords.keys():
        # check if the atom is in the structure
        # if not add zeros for the vector
        if len(pdb.coords[atom_type]) != 0:
            ind = pdb.kdtrees[atom_type].query_radius(mol.coords, r = 12)

            atom_type_list = []
            for i in range(mol.coords.shape[0]):
                
                atom_env_coords = pdb.coords[atom_type][ind[i], :]
                atom_lig_coords = mol.coords[i]
                
                #print(atom_type, mol.symbols[i], len(ind[i]))
                
                if atom_type == "C":
                    if mol.symbols[i] == "C":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[7] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[8] += len(ind[i])
                elif atom_type == "O":
                    if mol.symbols[i] == "C":
                        desc[9] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[10] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[11] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[12] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[13] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[14] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[15] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[16] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[17] += len(ind[i])
                elif atom_type == "N":
                    if mol.symbols[i] == "C":
                        desc[18] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[19] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[20] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[21] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[22] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[23] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[24] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[25] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[26] += len(ind[i])
                elif atom_type == "S":
                    if mol.symbols[i] == "C":
                        desc[27] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[28] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[29] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[30] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[31] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[32] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[33] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[34] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[35] += len(ind[i])
                elif atom_type == "P":
                    if mol.symbols[i] == "C":
                        desc[36] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[37] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[38] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[39] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[40] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[41] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[42] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[43] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[44] += len(ind[i])
                elif atom_type == "M1":
                    if mol.symbols[i] == "C":
                        desc[45] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[46] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[47] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[48] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[49] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[50] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[51] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[52] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[53] += len(ind[i])
                elif atom_type == "M2":
                    if mol.symbols[i] == "C":
                        desc[54] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[55] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[56] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[57] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[58] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[59] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[60] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[61] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[62] += len(ind[i])
    return desc

def compute(pdb_id):
    desc_arr = []
    mol_names_arr = []
    for i in range(1, 100):
        try:
            mol_names_arr.append(pdb_id + "_lig_{}".format(i))
            #print(pdb_id, "molnamesarr", mol_names_arr[-1])
            mol = read_xyz("./all/" + pdb_id + "/" + pdb_id + "_lig_{}.xyz".format(i))
            pdb = read_pdb("./all/" + pdb_id + "/" + pdb_id + "_prot_1.pdb")

            desc = RF_desc(mol, pdb)
            desc_arr.append(desc)
            
        except Exception as e:
            print("COMPUTE FINISHED", e)
            break
    return desc_arr, mol_names_arr


def main(): 
    data = pd.read_csv("./index_all.csv")
    codes = [code for code in list(Counter(data.docked_pdb).keys()) if len(code) == 4]
    X = []
    y = []
    try:
        for code in codes:
            desc_arr, mol_names_arr = compute(code)
            print("code, len", code, len(desc_arr), '\n')
            if desc_arr is not None:
                for desc_idx in range(len(desc_arr)):
                    desc_i = desc_arr[desc_idx]
                    mol_name_cur = mol_names_arr[desc_idx]
                    #print("DF SIZE:", mol_name_cur, len(data[data.pose_filename == mol_name_cur]))
                    X.append(desc_i)
                    y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                    #print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
            else:
                X.append(desc_i)
                y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
    except Exception as e:
        print("error in ", code, mol_name_cur,e)

    X = np.array(X).astype(int)
    y = np.array(y)

    np.save("rf_X_all.npy", X)
    np.save("rf_y_all.npy", y)
        

if __name__ == "__main__":
    main()

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1xal/1xal_lig_4.xyz'
code, len 1xal 3 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/2yme/2yme_lig_2.xyz'
code, len 2yme 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3uzc/3uzc_lig_11.xyz'
code, len 3uzc 10 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/4ug2/4ug2_lig_2.xyz'
code, len 4ug2 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/2ydv/2ydv_lig_6.xyz'
code, len 2ydv 5 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3qak/3qak_lig_2.xyz'
code, len 3qak 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3iar/3iar_lig_2.xyz'
code, len 3iar 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/5iz6/5iz6_lig_2.xyz'
code, len 5iz6 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/5z8h/5z8h_lig_2.xyz'
code, len 5z8h 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1kvl/1kvl_lig_4.xyz'
code, l

COMPUTE FINISHED [Errno 2] No such file or directory: './all/4dru/4dru_lig_9.xyz'
code, len 4dru 8 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3rze/3rze_lig_2.xyz'
code, len 3rze 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1qbs/1qbs_lig_3.xyz'
code, len 1qbs 2 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1ebw/1ebw_lig_13.xyz'
code, len 1ebw 12 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1ajv/1ajv_lig_5.xyz'
code, len 1ajv 4 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/4njv/4njv_lig_3.xyz'
code, len 4njv 2 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1ec2/1ec2_lig_7.xyz'
code, len 1ec2 6 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ekv/3ekv_lig_2.xyz'
code, len 3ekv 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1d4h/1d4h_lig_6.xyz'
code, len 1d4h 5 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1eby/1eby_lig_2.xyz'
code, l

In [11]:
read_pdb('./all/1ajv/1ajv_prot_1.pdb')[1]

{'C': <sklearn.neighbors.kd_tree.KDTree at 0x25fa8320098>,
 'O': <sklearn.neighbors.kd_tree.KDTree at 0x25fa8321d78>,
 'N': <sklearn.neighbors.kd_tree.KDTree at 0x25fa8322248>,
 'S': <sklearn.neighbors.kd_tree.KDTree at 0x25fa8320f08>}

In [2]:
#RFC12A

import numpy as np
import pandas as pd
from collections import namedtuple
from math import log10
from copy import copy, deepcopy
from sklearn.neighbors import KDTree
import os
import sys
np.set_printoptions(threshold=sys.maxsize)


Mol = namedtuple('Mol', ['symbols', 'coords', 'message'])

#symbols is a list of atomic name to be met in the pdb
#kdtrees - is a dictionary of the KD-trees for each element

PDB = namedtuple('PDB', ['symbols', 'kdtrees', 'coords'])


def read_xyz(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    message = data[1].rstrip("\n")
    symbols = []
    coords = np.array([]).reshape((0, 3)).astype(float)
    for item in data[2:]:
        if item[0] != "H":
            item = item.strip("\n")
            item = item.split(" ")
            item = list(filter(None, item))
            symbols.append(item[0])
            coords = np.concatenate((coords, np.array(item[1:]).reshape((1,3)).astype(float)), 0)
    mol = Mol(symbols = symbols, coords = coords, message = message)
    mol.coords.astype(float)
    return mol


def read_pdb(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    # retain only lines containing atomic coordinates
    data = [line.rstrip("\n") for line in data if line.startswith("ATOM") or line.startswith("HETATM")]
    symbols = ['C', 'O', 'N', 'S', 'P', 'M1', 'M2']
    coords = {'C': [], 'O': [], 'N': [], 'S': [], 'P': [], 'M1': [], 'M2': []}
    # read and keep coordinate per atom type
    for line in data:
        x = float(line[30:37])
        y = float(line[38:45])
        z = float(line[46:53])

        if len(line) < 80 or line[76:78] == "  ":
            if line.startswith("ATOM") or line[17:20] in ["CSO", "IAS", "PTR", "TPO", "SEP", "ACE", "PCA", "CSD", "LLP", "KCX"]:
                symbol = deepcopy(line[12:16]).strip()[0]
            elif line[17:20] == "MSE":
                symbol = deepcopy(line[12:16]).strip()
                if symbol != "SE":
                    symbol = symbol[0]
            else:
                symbol = deepcopy(line[12:16]).strip()

        elif line[76] == " ":
            symbol = line[77]

        else:
            symbol = line[76:78]

        if symbol in ['C', 'C1+', 'O', 'O1-', 'N', 'N1+', 'S', 'P', 'Cl']:
            
            if len(symbol) != 1:
                symbol = symbol[0]
            coords[symbol].append([x, y, z])
        elif symbol in ['LI', 'NA', 'K', 'RB', 'CS']:
            coords['M1'].append([x, y, z])
        elif symbol in ['AU', 'BA', 'CA', 'CD', 'SR', 'BE', 'CO', 'MG', 'CU', 'FE', 'HG', 'MN', 'NI', 'ZN' , 'Zn2+' , 'Zn']:
            coords['M2'].append([x, y, z])
        elif symbol == 'SE':
            coords['S'].append([x, y, z])
        elif symbol == 'H':
            continue
        else:
            print(symbol, line[0:78])
            raise ValueError("Unknown atomic symbol in pdb")

    kdtrees = {}

    # create a dict of KD trees
    for atom_type in symbols:
        coords[atom_type] = np.array(coords[atom_type])
        if len(coords[atom_type]) != 0:
            kdtrees[atom_type] = KDTree(coords[atom_type], leaf_size=10)

    pdb = PDB(symbols=symbols, kdtrees=kdtrees, coords=coords)

    return pdb

 
     

def RF_desc(mol, pdb):

    desc = [0.] * 7
    for atom_type in pdb.coords.keys():
        # check if the atom is in the structure
        # if not add zeros for the vector
        if len(pdb.coords[atom_type]) != 0:
            ind = pdb.kdtrees[atom_type].query_radius(mol.coords, r = 12)

            atom_type_list = []
            for i in range(mol.coords.shape[0]):
                
                atom_env_coords = pdb.coords[atom_type][ind[i], :]
                atom_lig_coords = mol.coords[i]
                
                #print(atom_type, mol.symbols[i], len(ind[i]))
                
                if atom_type == "C":
                    if mol.symbols[i] == "C":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[0] += len(ind[i])
                elif atom_type == "O":
                    if mol.symbols[i] == "C":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[1] += len(ind[i])
                elif atom_type == "N":
                    if mol.symbols[i] == "C":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[2] += len(ind[i])
                elif atom_type == "S":
                    if mol.symbols[i] == "C":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[3] += len(ind[i])
                elif atom_type == "P":
                    if mol.symbols[i] == "C":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[4] += len(ind[i])
                elif atom_type == "M1":
                    if mol.symbols[i] == "C":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[5] += len(ind[i])
                elif atom_type == "M2":
                    if mol.symbols[i] == "C":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[6] += len(ind[i])
    return desc

def compute(pdb_id):
    desc_arr = []
    mol_names_arr = []
    for i in range(1, 100):
        try:
            mol_names_arr.append(pdb_id + "_lig_{}".format(i))
            #print(pdb_id, "molnamesarr", mol_names_arr[-1])
            mol = read_xyz("./all/" + pdb_id + "/" + pdb_id + "_lig_{}.xyz".format(i))
            pdb = read_pdb("./all/" + pdb_id + "/" + pdb_id + "_prot_1.pdb")

            desc = RF_desc(mol, pdb)
            desc_arr.append(desc)
            
        except Exception as e:
            print("COMPUTE FINISHED", e)
            break
    return desc_arr, mol_names_arr


def main(): 
    data = pd.read_csv("./index_all.csv")
    codes = [code for code in list(Counter(data.docked_pdb).keys()) if len(code) == 4]
    X = []
    y = []
    try:
        for code in codes:
            desc_arr, mol_names_arr = compute(code)
            print("code, len", code, len(desc_arr), '\n')
            if desc_arr is not None:
                for desc_idx in range(len(desc_arr)):
                    desc_i = desc_arr[desc_idx]
                    mol_name_cur = mol_names_arr[desc_idx]
                    #print("DF SIZE:", mol_name_cur, len(data[data.pose_filename == mol_name_cur]))
                    X.append(desc_i)
                    y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                    #print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
            else:
                X.append(desc_i)
                y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
    except Exception as e:
        print("error in ", code, mol_name_cur,e)

    X = np.array(X).astype(int)
    y = np.array(y)

    np.save("rfc_X_all.npy", X)
    np.save("rfc_y_all.npy", y)
        

if __name__ == "__main__":
    main()

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1xal/1xal_lig_4.xyz'
code, len 1xal 3 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/2yme/2yme_lig_2.xyz'
code, len 2yme 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3uzc/3uzc_lig_11.xyz'
code, len 3uzc 10 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/4ug2/4ug2_lig_2.xyz'
code, len 4ug2 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/2ydv/2ydv_lig_6.xyz'
code, len 2ydv 5 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3qak/3qak_lig_2.xyz'
code, len 3qak 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3iar/3iar_lig_2.xyz'
code, len 3iar 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/5iz6/5iz6_lig_2.xyz'
code, len 5iz6 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/5z8h/5z8h_lig_2.xyz'
code, len 5z8h 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1kvl/1kvl_lig_4.xyz'
code, l

COMPUTE FINISHED [Errno 2] No such file or directory: './all/4dru/4dru_lig_9.xyz'
code, len 4dru 8 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3rze/3rze_lig_2.xyz'
code, len 3rze 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1qbs/1qbs_lig_3.xyz'
code, len 1qbs 2 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1ebw/1ebw_lig_13.xyz'
code, len 1ebw 12 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1ajv/1ajv_lig_5.xyz'
code, len 1ajv 4 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/4njv/4njv_lig_3.xyz'
code, len 4njv 2 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1ec2/1ec2_lig_7.xyz'
code, len 1ec2 6 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ekv/3ekv_lig_2.xyz'
code, len 3ekv 1 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1d4h/1d4h_lig_6.xyz'
code, len 1d4h 5 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1eby/1eby_lig_2.xyz'
code, l

In [16]:
#SS12A

import numpy as np
import pandas as pd
from collections import namedtuple
from math import log10
from copy import copy, deepcopy
from sklearn.neighbors import KDTree
from collections import Counter
import os
import sys
np.set_printoptions(threshold=sys.maxsize)
from Bio import SeqUtils


Mol = namedtuple('Mol', ['symbols', 'coords', 'message'])

#symbols is a list of atomic name to be met in the pdb
#kdtrees - is a dictionary of the KD-trees for each atom

PDB = namedtuple('PDB', ['symbols', 'kdtrees', 'coords'])

#res_types is a dictionary of residue information (name, chain, number) and its secondary structure info

Sec = namedtuple('Sec', ['res_types'])


def read_xyz(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    message = data[1].rstrip("\n")
    symbols = []
    coords = np.array([]).reshape((0, 3)).astype(float)
    for item in data[2:]:
        if item[0] != "H":
            item = item.strip("\n")
            item = item.split(" ")
            item = list(filter(None, item))
            symbols.append(item[0])
            coords = np.concatenate((coords, np.array(item[1:]).reshape((1,3)).astype(float)), 0)
    mol = Mol(symbols = symbols, coords = coords, message = message)
    mol.coords.astype(float)
    return mol


def read_pdb(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    #retain only lines containing atomic coordinates
    data = [line.rstrip("\n") for line in data if line.startswith("ATOM") or line.startswith("HETATM")]


    xyz = []
    symbols = []

    for lines in data:  
        if (lines[11:17].strip(' ')[0]) != ('H') and (lines[11:17].strip(' ')[0]) not in ('K'):
            if ((((lines[71:80])).strip(' ')[:2]).upper()) not in  ('LI', 'NA', 'RB', 'CS', 
                                                     'AU', 'BA', 'CA', 'CD', 'SR', 
                                                     'BE', 'CO', 'MG', 'CU', 'FE', 
                                                     'HG', 'MN', 'NI', 'ZN'):
                    symbols.append((lines[16:20].strip(' '), lines[21:22], lines[23:26].strip(' ')))
                    x = float(lines[30:37])
                    y = float(lines[38:45])
                    z = float(lines[46:53])
                    xyz.append([x, y, z])

    res_coord_pair = []

    for i in range(len(symbols)):
        res_coord_pair.append((symbols[i], xyz[i]))


    unique_resnames = (list(Counter(symbols).keys()))

    coords = {}

    for res in res_coord_pair:
        res_cur = res[0]
        coord_cur = res[1]
        if res_cur not in coords.keys():
            coords[res_cur] = []
        coords[res_cur].append(coord_cur)

    kdtrees = {}

    #create a dict of KD trees
    for res in coords.keys():
        #coords[res] = np.array(coords[res])
        kdtrees[res] = KDTree(coords[res], leaf_size=10)

    pdb = PDB(symbols=unique_resnames, kdtrees=kdtrees, coords=coords)
    return pdb

 
def read_dssp(filename):
    f = open(filename, 'r')
    readl = f.readlines()
    f.close()
    dssps = [line.rstrip("\n") for line in readl[28:]]    

    ssr = []
    for lines in dssps:
            ssr.append((lines[13:14], lines[11:13].strip(' '), lines[7:11].strip(' '), lines[16:17]))

    ss=[]
    for lines in ssr:
        if lines != ('!', '', '', ' '):
            
            ss.append(lines)

    residues=[]
    
    resnames=[]
    reschains=[]
    resnums=[]

    for n in range(len(ss)):
        resnames.append((str(SeqUtils.IUPACData.protein_letters_1to3.get(ss[n][0]))).upper())
        reschains.append(ss[n][1])
        resnums.append(ss[n][2])
        residues.append((resnames[n], reschains[n], resnums[n]))
    
    restypes=[]
    
    for n in range(len(ss)):
        for i in (ss[n][3]):
            if i == (' '):
                i = 'R'
            restypes.append(i)
            
    # restypes is a dict of residues vs types
    res_types = {}

    res_types = dict(zip(residues, restypes))

    sec = Sec(res_types = res_types)
    return sec     


def SS_desc(mol, pdb, sec):
    desc = [0.] * 81
    for atom_type in pdb.coords.keys():
        # check if the residue is in the structure
        # if not add zeros for the vector
            ind = pdb.kdtrees[atom_type].query_radius(mol.coords, r = 12)
            counts = ind
            for i in range(len(ind)):
                if len(counts[i]) != 0:
                    counts[i] = np.array(1)
                else:
                    counts[i] = np.array(0)
            #for i in range(mol.coords.shape[0]):

                #print(atom_type, mol.symbols[i], len(ind[i]))

                if sec.res_types.get(atom_type) == "H":
                    if mol.symbols[i] == "C":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[3] = desc[3] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[4] = desc[4] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[5] = desc[5] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[6] = desc[6] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[7] = desc[7] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[8] = desc[8] + counts[i]

                if sec.res_types.get(atom_type) == "B":
                    if mol.symbols[i] == "C":
                        desc[9] = desc[9] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[10] = desc[10] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[11] = desc[11] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[12] = desc[12] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[13] = desc[13] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[14] = desc[14] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[15] = desc[15] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[16] = desc[16] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[17] = desc[17] + counts[i]

                if sec.res_types.get(atom_type) == "E":
                    if mol.symbols[i] == "C":
                        desc[18] = desc[18] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[19] = desc[19] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[20] = desc[20] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[21] = desc[21] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[22] = desc[22] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[23] = desc[23] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[24] = desc[24] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[25] = desc[25] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[26] = desc[26] + counts[i]

                if sec.res_types.get(atom_type) == "G":
                    if mol.symbols[i] == "C":
                        desc[27] = desc[27] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[28] = desc[28] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[29] = desc[29] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[30] = desc[30] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[31] = desc[31] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[32] = desc[32] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[33] = desc[33] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[34] = desc[34] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[35] = desc[35] + counts[i]

                if sec.res_types.get(atom_type) == "I":
                    if mol.symbols[i] == "C":
                        desc[36] = desc[36] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[37] = desc[37] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[38] = desc[38] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[39] = desc[39] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[40] = desc[40] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[41] = desc[41] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[42] = desc[42] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[43] = desc[43] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[44] = desc[44] + counts[i]

                if sec.res_types.get(atom_type) == "T":
                    if mol.symbols[i] == "C":
                        desc[45] = desc[45] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[46] = desc[46] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[47] = desc[47] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[48] = desc[48] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[49] = desc[49] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[50] = desc[50] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[51] = desc[51] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[52] = desc[52] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[53] = desc[53] + counts[i]

                if sec.res_types.get(atom_type) == "T":
                    if mol.symbols[i] == "C":
                        desc[54] = desc[54] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[55] = desc[55] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[56] = desc[56] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[57] = desc[57] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[58] = desc[58] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[59] = desc[59] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[60] = desc[60] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[61] = desc[61] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[62] = desc[62] + counts[i]

                if sec.res_types.get(atom_type) == "S":
                    if mol.symbols[i] == "C":
                        desc[63] = desc[63] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[64] = desc[64] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[65] = desc[65] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[66] = desc[66] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[67] = desc[67] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[68] = desc[68] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[69] = desc[69] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[70] = desc[70] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[71] = desc[71] + counts[i]

                if sec.res_types.get(atom_type) == "R":
                    if mol.symbols[i] == "C":
                        desc[72] = desc[72] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[73] = desc[73] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[74] = desc[74] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[75] = desc[75] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[76] = desc[76] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[77] = desc[77] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[78] = desc[78] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[79] = desc[79] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[80] = desc[80] + counts[i]
    return desc

def compute(pdb_id):
    desc_arr = []
    mol_names_arr = []
    for i in range(1, 100):
        try:
            mol_names_arr.append(pdb_id + "_lig_{}".format(i))
            #print(pdb_id, "molnamesarr", mol_names_arr[-1])
            mol = read_xyz("./all/" + pdb_id + "/" + pdb_id + "_lig_{}.xyz".format(i))
            pdb = read_pdb("./all/" + pdb_id + "/" + pdb_id + "_prot_1.pdb")
            sec = read_dssp("./all/" + pdb_id + "/" + pdb_id + "_sec_1.dssp")

            desc = SS_desc(mol, pdb, sec)
            desc_arr.append(desc)

        except Exception as e:
            print("COMPUTE FINISHED", e)
            break
    return desc_arr, mol_names_arr


def main(): 
    data = pd.read_csv("./index_all.csv")
    codes = [code for code in list(Counter(data.docked_pdb).keys()) if len(code) == 4]
    X = []
    y = []
    try:
        for code in codes:
            desc_arr, mol_names_arr = compute(code)
            print("code, len", code, len(desc_arr), '\n')
            if desc_arr is not None:
                for desc_idx in range(len(desc_arr)):
                    desc_i = desc_arr[desc_idx]
                    mol_name_cur = mol_names_arr[desc_idx]
                    print("DF SIZE:", mol_name_cur, len(data[data.pose_filename == mol_name_cur]))
                    X.append(desc_i)
                    y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                    #print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
            else:
                X.append(desc_i)
                y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
    except Exception as e:
        print("error in ", code, mol_name_cur,e)

    X = np.array(X).astype(int)
    y = np.array(y).astype(float)

    np.save("ss_X_all.npy", X)
    np.save("ss_y_allnpy", y)
        

if __name__ == "__main__":
    main()

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1xal/1xal_lig_4.xyz'
code, len 1xal 3 

DF SIZE: 1xal_lig_1 1
DF SIZE: 1xal_lig_2 1
DF SIZE: 1xal_lig_3 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/2yme/2yme_lig_2.xyz'
code, len 2yme 1 

DF SIZE: 2yme_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3uzc/3uzc_lig_11.xyz'
code, len 3uzc 10 

DF SIZE: 3uzc_lig_1 1
DF SIZE: 3uzc_lig_2 1
DF SIZE: 3uzc_lig_3 1
DF SIZE: 3uzc_lig_4 1
DF SIZE: 3uzc_lig_5 1
DF SIZE: 3uzc_lig_6 1
DF SIZE: 3uzc_lig_7 1
DF SIZE: 3uzc_lig_8 1
DF SIZE: 3uzc_lig_9 1
DF SIZE: 3uzc_lig_10 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/4ug2/4ug2_lig_2.xyz'
code, len 4ug2 1 

DF SIZE: 4ug2_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/2ydv/2ydv_lig_6.xyz'
code, len 2ydv 5 

DF SIZE: 2ydv_lig_1 1
DF SIZE: 2ydv_lig_2 1
DF SIZE: 2ydv_lig_3 1
DF SIZE: 2ydv_lig_4 1
DF SIZE: 2ydv_lig_5 1
COMPUTE FINISHED [Errno 2] No such file or directory

COMPUTE FINISHED [Errno 2] No such file or directory: './all/2aay/2aay_lig_3.xyz'
code, len 2aay 2 

DF SIZE: 2aay_lig_1 1
DF SIZE: 2aay_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3nrc/3nrc_lig_6.xyz'
code, len 3nrc 5 

DF SIZE: 3nrc_lig_1 1
DF SIZE: 3nrc_lig_2 1
DF SIZE: 3nrc_lig_3 1
DF SIZE: 3nrc_lig_4 1
DF SIZE: 3nrc_lig_5 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/4no7/4no7_lig_3.xyz'
code, len 4no7 2 

DF SIZE: 4no7_lig_1 1
DF SIZE: 4no7_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3f9m/3f9m_lig_3.xyz'
code, len 3f9m 2 

DF SIZE: 3f9m_lig_1 1
DF SIZE: 3f9m_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/1jyq/1jyq_lig_3.xyz'
code, len 1jyq 2 

DF SIZE: 1jyq_lig_1 1
DF SIZE: 1jyq_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ldo/3ldo_lig_2.xyz'
code, len 3ldo 1 

DF SIZE: 3ldo_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ldp/3ldp_lig_8.xyz'
code

COMPUTE FINISHED [Errno 2] No such file or directory: './all/6f1n/6f1n_lig_2.xyz'
code, len 6f1n 1 

DF SIZE: 6f1n_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/6elo/6elo_lig_3.xyz'
code, len 6elo 2 

DF SIZE: 6elo_lig_1 1
DF SIZE: 6elo_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/5j64/5j64_lig_5.xyz'
code, len 5j64 4 

DF SIZE: 5j64_lig_1 1
DF SIZE: 5j64_lig_2 1
DF SIZE: 5j64_lig_3 1
DF SIZE: 5j64_lig_4 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/6eln/6eln_lig_2.xyz'
code, len 6eln 1 

DF SIZE: 6eln_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/5j20/5j20_lig_3.xyz'
code, len 5j20 2 

DF SIZE: 5j20_lig_1 1
DF SIZE: 5j20_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/5j86/5j86_lig_9.xyz'
code, len 5j86 8 

DF SIZE: 5j86_lig_1 1
DF SIZE: 5j86_lig_2 1
DF SIZE: 5j86_lig_3 1
DF SIZE: 5j86_lig_4 1
DF SIZE: 5j86_lig_5 1
DF SIZE: 5j86_lig_6 1
DF SIZE: 5j86_lig_7 1
DF SIZE: 5j86_lig_8 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/5if1/5if1_lig_4.xyz'
code, len 5if1 3 

DF SIZE: 5if1_lig_1 1
DF SIZE: 5if1_lig_2 1
DF SIZE: 5if1_lig_3 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/1w98/1w98_lig_4.xyz'
code, len 1w98 3 

DF SIZE: 1w98_lig_1 1
DF SIZE: 1w98_lig_2 1
DF SIZE: 1w98_lig_3 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ddu/3ddu_lig_4.xyz'
code, len 3ddu 3 

DF SIZE: 3ddu_lig_1 1
DF SIZE: 3ddu_lig_2 1
DF SIZE: 3ddu_lig_3 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/1bil/1bil_lig_8.xyz'
code, len 1bil 7 

DF SIZE: 1bil_lig_1 1
DF SIZE: 1bil_lig_2 1
DF SIZE: 1bil_lig_3 1
DF SIZE: 1bil_lig_4 1
DF SIZE: 1bil_lig_5 1
DF SIZE: 1bil_lig_6 1
DF SIZE: 1bil_lig_7 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/2v0z/2v0z_lig_2.xyz'
code, len 2v0z 1 

DF SIZE: 2v0z_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/6c4d/6c4d_lig_34.xyz'
code, len 6c4d 33 

DF SIZE: 6c4d_lig_

In [ ]:
#SSC12A

import numpy as np
import pandas as pd
from collections import namedtuple
from math import log10
from copy import copy, deepcopy
from sklearn.neighbors import KDTree
from collections import Counter
import os
import sys
np.set_printoptions(threshold=sys.maxsize)
from Bio import SeqUtils


Mol = namedtuple('Mol', ['symbols', 'coords', 'message'])

#symbols is a list of atomic name to be met in the pdb
#kdtrees - is a dictionary of the KD-trees for each atom

PDB = namedtuple('PDB', ['symbols', 'kdtrees', 'coords'])

#res_types is a dictionary of residue information (name, chain, number) and its secondary structure info

Sec = namedtuple('Sec', ['res_types'])


def read_xyz(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    message = data[1].rstrip("\n")
    symbols = []
    coords = np.array([]).reshape((0, 3)).astype(float)
    for item in data[2:]:
        if item[0] != "H":
            item = item.strip("\n")
            item = item.split(" ")
            item = list(filter(None, item))
            symbols.append(item[0])
            coords = np.concatenate((coords, np.array(item[1:]).reshape((1,3)).astype(float)), 0)
    mol = Mol(symbols = symbols, coords = coords, message = message)
    mol.coords.astype(float)
    return mol


def read_pdb(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    #retain only lines containing atomic coordinates
    data = [line.rstrip("\n") for line in data if line.startswith("ATOM") or line.startswith("HETATM")]


    xyz = []
    symbols = []

    for lines in data:  
        if (lines[11:17].strip(' ')[0]) != ('H') and (lines[11:17].strip(' ')[0]) not in ('K'):
            if ((((lines[71:80])).strip(' ')[:2]).upper()) not in  ('LI', 'NA', 'RB', 'CS', 
                                                     'AU', 'BA', 'CA', 'CD', 'SR', 
                                                     'BE', 'CO', 'MG', 'CU', 'FE', 
                                                     'HG', 'MN', 'NI', 'ZN'):
                    symbols.append((lines[16:20].strip(' '), lines[21:22], lines[23:26].strip(' ')))
                    x = float(lines[30:37])
                    y = float(lines[38:45])
                    z = float(lines[46:53])
                    xyz.append([x, y, z])

    res_coord_pair = []

    for i in range(len(symbols)):
        res_coord_pair.append((symbols[i], xyz[i]))


    unique_resnames = (list(Counter(symbols).keys()))

    coords = {}

    for res in res_coord_pair:
        res_cur = res[0]
        coord_cur = res[1]
        if res_cur not in coords.keys():
            coords[res_cur] = []
        coords[res_cur].append(coord_cur)

    kdtrees = {}

    #create a dict of KD trees
    for res in coords.keys():
        #coords[res] = np.array(coords[res])
        kdtrees[res] = KDTree(coords[res], leaf_size=10)

    pdb = PDB(symbols=unique_resnames, kdtrees=kdtrees, coords=coords)
    return pdb

 
def read_dssp(filename):
    f = open(filename, 'r')
    readl = f.readlines()
    f.close()
    dssps = [line.rstrip("\n") for line in readl[28:]]    

    ssr = []
    for lines in dssps:
            ssr.append((lines[13:14], lines[11:13].strip(' '), lines[7:11].strip(' '), lines[16:17]))

    ss=[]
    for lines in ssr:
        if lines != ('!', '', '', ' '):
            
            ss.append(lines)

    residues=[]
    
    resnames=[]
    reschains=[]
    resnums=[]

    for n in range(len(ss)):
        resnames.append((str(SeqUtils.IUPACData.protein_letters_1to3.get(ss[n][0]))).upper())
        reschains.append(ss[n][1])
        resnums.append(ss[n][2])
        residues.append((resnames[n], reschains[n], resnums[n]))
    
    restypes=[]
    
    for n in range(len(ss)):
        for i in (ss[n][3]):
            if i == (' '):
                i = 'R'
            restypes.append(i)
            
    # restypes is a dict of residues vs types
    res_types = {}

    res_types = dict(zip(residues, restypes))

    sec = Sec(res_types = res_types)
    return sec     


def SS_desc(mol, pdb, sec):
    desc = [0.] * 9
    for atom_type in pdb.coords.keys():
        # check if the residue is in the structure
        # if not add zeros for the vector
            ind = pdb.kdtrees[atom_type].query_radius(mol.coords, r = 12)
            counts = ind
            for i in range(len(ind)):
                if len(counts[i]) != 0:
                    counts[i] = np.array(1)
                else:
                    counts[i] = np.array(0)
            #for i in range(mol.coords.shape[0]):

                #print(atom_type, mol.symbols[i], len(ind[i]))

                if sec.res_types.get(atom_type) == "H":
                    if mol.symbols[i] == "C":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[0] = desc[0] + counts[i]

                if sec.res_types.get(atom_type) == "B":
                    if mol.symbols[i] == "C":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[1] = desc[1] + counts[i]

                if sec.res_types.get(atom_type) == "E":
                    if mol.symbols[i] == "C":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[2] = desc[2] + counts[i]

                if sec.res_types.get(atom_type) == "G":
                    if mol.symbols[i] == "C":
                        desc[3] = desc[3] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[3] = desc[3] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[3] = desc[3] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[3] = desc[3] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[3] = desc[3] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[3] = desc[3] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[3] = desc[3] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[3] = desc[3] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[3] = desc[3] + counts[i]

                if sec.res_types.get(atom_type) == "I":
                    if mol.symbols[i] == "C":
                        desc[4] = desc[4] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[4] = desc[4] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[4] = desc[4] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[4] = desc[4] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[4] = desc[4] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[4] = desc[4] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[4] = desc[4] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[4] = desc[4] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[4] = desc[4] + counts[i]

                if sec.res_types.get(atom_type) == "T":
                    if mol.symbols[i] == "C":
                        desc[5] = desc[5] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[5] = desc[5] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[5] = desc[5] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[5] = desc[5] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[5] = desc[5] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[5] = desc[5] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[5] = desc[5] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[5] = desc[5] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[5] = desc[5] + counts[i]

                if sec.res_types.get(atom_type) == "T":
                    if mol.symbols[i] == "C":
                        desc[6] = desc[6] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[6] = desc[6] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[6] = desc[6] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[6] = desc[6] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[6] = desc[6] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[6] = desc[6] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[6] = desc[6] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[6] = desc[6] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[6] = desc[6] + counts[i]

                if sec.res_types.get(atom_type) == "S":
                    if mol.symbols[i] == "C":
                        desc[7] = desc[7] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[7] = desc[7] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[7] = desc[7] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[7] = desc[7] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[7] = desc[7] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[7] = desc[7] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[7] = desc[7] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[7] = desc[7] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[7] = desc[7] + counts[i]

                if sec.res_types.get(atom_type) == "R":
                    if mol.symbols[i] == "C":
                        desc[8] = desc[8] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[8] = desc[8] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[8] = desc[8] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[8] = desc[8] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[8] = desc[8] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[8] = desc[8] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[8] = desc[8] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[8] = desc[8] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[8] = desc[8] + counts[i]
    return desc

def compute(pdb_id):
    desc_arr = []
    mol_names_arr = []
    for i in range(1, 100):
        try:
            mol_names_arr.append(pdb_id + "_lig_{}".format(i))
            #print(pdb_id, "molnamesarr", mol_names_arr[-1])
            mol = read_xyz("./all/" + pdb_id + "/" + pdb_id + "_lig_{}.xyz".format(i))
            pdb = read_pdb("./all/" + pdb_id + "/" + pdb_id + "_prot_1.pdb")
            sec = read_dssp("./all/" + pdb_id + "/" + pdb_id + "_sec_1.dssp")

            desc = SS_desc(mol, pdb, sec)
            desc_arr.append(desc)

        except Exception as e:
            print("COMPUTE FINISHED", e)
            break
    return desc_arr, mol_names_arr


def main(): 
    data = pd.read_csv("./index_all.csv")
    codes = [code for code in list(Counter(data.docked_pdb).keys()) if len(code) == 4]
    X = []
    y = []
    z = []
    t = []
    try:
        for code in codes:
            desc_arr, mol_names_arr = compute(code)
            print("code, len", code, len(desc_arr), '\n')
            if desc_arr is not None:
                for desc_idx in range(len(desc_arr)):
                    desc_i = desc_arr[desc_idx]
                    mol_name_cur = mol_names_arr[desc_idx]
                    print("DF SIZE:", mol_name_cur, len(data[data.pose_filename == mol_name_cur]))
                    X.append(desc_i)
                    y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                    z.append(data[data.pose_filename == mol_name_cur].smiles.iloc[0])
                    t.append(data[data.pose_filename == mol_name_cur].target.iloc[0])
                    #print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
            else:
                X.append(desc_i)
                y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                z.append(data[data.pose_filename == mol_name_cur].smiles.iloc[0])
                t.append(data[data.pose_filename == mol_name_cur].target.iloc[0])

    except Exception as e:
        print("error in ", code, mol_name_cur,e)

    X = np.array(X).astype(int)
    y = np.array(y).astype(float)
    z = np.array(z)
    t = np.array(t)


    np.save("ssc_X_all.npy", X)
    np.save("ssc_y_all.npy", y)
    np.save("smi_all.npy", z)
    np.save("target_all.npy", t)

    
if __name__ == "__main__":
    main()

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1xal/1xal_lig_4.xyz'
code, len 1xal 3 

DF SIZE: 1xal_lig_1 1
DF SIZE: 1xal_lig_2 1
DF SIZE: 1xal_lig_3 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/2yme/2yme_lig_2.xyz'
code, len 2yme 1 

DF SIZE: 2yme_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3uzc/3uzc_lig_11.xyz'
code, len 3uzc 10 

DF SIZE: 3uzc_lig_1 1
DF SIZE: 3uzc_lig_2 1
DF SIZE: 3uzc_lig_3 1
DF SIZE: 3uzc_lig_4 1
DF SIZE: 3uzc_lig_5 1
DF SIZE: 3uzc_lig_6 1
DF SIZE: 3uzc_lig_7 1
DF SIZE: 3uzc_lig_8 1
DF SIZE: 3uzc_lig_9 1
DF SIZE: 3uzc_lig_10 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/4ug2/4ug2_lig_2.xyz'
code, len 4ug2 1 

DF SIZE: 4ug2_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/2ydv/2ydv_lig_6.xyz'
code, len 2ydv 5 

DF SIZE: 2ydv_lig_1 1
DF SIZE: 2ydv_lig_2 1
DF SIZE: 2ydv_lig_3 1
DF SIZE: 2ydv_lig_4 1
DF SIZE: 2ydv_lig_5 1
COMPUTE FINISHED [Errno 2] No such file or directory

COMPUTE FINISHED [Errno 2] No such file or directory: './all/2aay/2aay_lig_3.xyz'
code, len 2aay 2 

DF SIZE: 2aay_lig_1 1
DF SIZE: 2aay_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3nrc/3nrc_lig_6.xyz'
code, len 3nrc 5 

DF SIZE: 3nrc_lig_1 1
DF SIZE: 3nrc_lig_2 1
DF SIZE: 3nrc_lig_3 1
DF SIZE: 3nrc_lig_4 1
DF SIZE: 3nrc_lig_5 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/4no7/4no7_lig_3.xyz'
code, len 4no7 2 

DF SIZE: 4no7_lig_1 1
DF SIZE: 4no7_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3f9m/3f9m_lig_3.xyz'
code, len 3f9m 2 

DF SIZE: 3f9m_lig_1 1
DF SIZE: 3f9m_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/1jyq/1jyq_lig_3.xyz'
code, len 1jyq 2 

DF SIZE: 1jyq_lig_1 1
DF SIZE: 1jyq_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ldo/3ldo_lig_2.xyz'
code, len 3ldo 1 

DF SIZE: 3ldo_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ldp/3ldp_lig_8.xyz'
code

In [5]:
#BB12A

import numpy as np
import pandas as pd
from collections import namedtuple
from math import cos, pi, log10
from copy import copy, deepcopy
from sklearn.neighbors import KDTree
import os
from multiprocessing import Pool


Mol = namedtuple('Mol', ['symbols', 'coords', 'message'])

#symbols is a list of atomic name to be met in the pdb
#kdtrees - is a dictionary of the KD-trees for each element

PDB = namedtuple('PDB', ['symbols', 'kdtrees', 'coords'])


def read_xyz(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    message = data[1].rstrip("\n")
    symbols = []
    coords = np.array([]).reshape((0, 3)).astype(float)
    for item in data[2:]:
        if item[0] != "H":
            item = item.strip("\n")
            item = item.split(" ")
            item = list(filter(None, item))
            symbols.append(item[0])
            coords = np.concatenate((coords, np.array(item[1:]).reshape((1,3)).astype(float)), 0)
    mol = Mol(symbols = symbols, coords = coords, message = message)
    mol.coords.astype(float)
    return mol


def read_pdb(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    # retain only lines containing atomic coordinates
    data = [line.rstrip("\n") for line in data if line.startswith("ATOM")]    
    symbols = ['Cb', 'Ob', 'Nb', 'Cs', 'Os', 'Ns', 'Ss']
    coords = {'Cb': [], 'Ob': [], 'Nb': [], 'Cs': [], 'Os': [], 'Ns': [], 'Ss': []}
    # read and keep coordinate per atom type
    for line in data:
        x = float(line[30:37])
        y = float(line[38:45])
        z = float(line[46:53])

        symbol = deepcopy(line[12:16]).strip()          
    
        if symbol in ['C', 'CA', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']:
            coords['Cb'].append([x, y, z])
       
        elif symbol in ['O', 'OA', 'O1', 'O2', 'O3', 'O4', 'O5', 'O6', 'O7', 'O8', 'O9', 'OXT']:
            coords['Ob'].append([x, y, z])
        
        elif symbol in ['N', 'NA', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9']:
            coords['Nb'].append([x, y, z])
         
        elif symbol in ['CB','CB1','CB2','CB3','CB4','CB5','CB6','CB7','CB8','CB9','CG','CG1','CG2','CG3','CG4','CG5','CG6','CG7','CG8','CG9','CD','CD1','CD2','CD3','CD4','CD5','CD6','CD7','CD8','CD9','CE','CE1','CE2','CE3','CE4','CE5','CE6','CE7','CE8','CE9','CZ','CZ1','CZ2','CZ3','CZ4','CZ5','CZ6','CZ7','CZ8','CZ9','CH','CH1','CH2','CH3','CH4','CH5','CH6','CH7','CH8','CH9']:
            coords['Cs'].append([x, y, z])
        
        elif symbol in ['OB','OB1','OB2','OB3','OB4','OB5','OB6','OB7','OB8','OB9','OG','OG1','OG2','OG3','OG4','OG5','OG6','OG7','OG8','OG9','OD','OD1','OD2','OD3','OD4','OD5','OD6','OD7','OD8','OD9','OE','OE1','OE2','OE3','OE4','OE5','OE6','OE7','OE8','OE9','OZ','OZ1','OZ2','OZ3','OZ4','OZ5','OZ6','OZ7','OZ8','OZ9','OH','OH1','OH2','OH3','OH4','OH5','OH6','OH7','OH8','OH9']:
            coords['Os'].append([x, y, z])
        
        elif symbol in ['NB','NB1','NB2','NB3','NB4','NB5','NB6','NB7','NB8','NB9','NG','NG1','NG2','NG3','NG4','NG5','NG6','NG7','NG8','NG9','ND','ND1','ND2','ND3','ND4','ND5','ND6','ND7','ND8','ND9','NE','NE1','NE2','NE3','NE4','NE5','NE6','NE7','NE8','NE9','NZ','NZ1','NZ2','NZ3','NZ4','NZ5','NZ6','NZ7','NZ8','NZ9','NH','NH1','NH2','NH3','NH4','NH5','NH6','NH7','NH8','NH9']:
            coords['Ns'].append([x, y, z])
        
        elif symbol in ['SB','SB1','SB2','SB3','SB4','SB5','SB6','SB7','SB8','SB9','SG','SG1','SG2','SG3','SG4','SG5','SG6','SG7','SG8','SG9','SD','SD1','SD2','SD3','SD4','SD5','SD6','SD7','SD8','SD9','SE','SE1','SE2','SE3','SE4','SE5','SE6','SE7','SE8','SE9','SZ','SZ1','SZ2','SZ3','SZ4','SZ5','SZ6','SZ7','SZ8','SZ9','SH','SH1','SH2','SH3','SH4','SH5','SH6','SH7','SH8','SH9', 'S']:
            coords['Ss'].append([x, y, z])
        
        elif symbol == 'SE':
            coords['Ss'].append([x, y, z])
        
        elif symbol in ['LI', 'NA', 'K', 'RB', 'CS', 'AU', 'BA', 'CA', 'CD', 'SR', 'BE', 'CO', 'MG', 'CU', 'FE', 'HG', 'MN', 'NI', 'ZN']:
            continue
        
        elif symbol.strip()[0] == 'H':
            continue
        
        else:
            print(symbol, line[:])
            raise ValueError("Unknown atomic symbol in pdb")

    kdtrees = {}

    # create a dict of KD trees
    for atom_type in symbols:
        coords[atom_type] = np.array(coords[atom_type])
        if len(coords[atom_type]) != 0:
            kdtrees[atom_type] = KDTree(coords[atom_type], leaf_size=10)

    mol = PDB(symbols=symbols, kdtrees=kdtrees, coords=coords)

    return mol

 
     

def BB_desc(mol, pdb):

    desc = [0.] * 63
    for atom_type in pdb.coords.keys():
        # check if the atom is in the structure
        # if not add zeros for the vector
        if len(pdb.coords[atom_type]) != 0:
            ind = pdb.kdtrees[atom_type].query_radius(mol.coords, r = 12)

            atom_type_list = []
            for i in range(mol.coords.shape[0]):
                
                atom_env_coords = pdb.coords[atom_type][ind[i], :]
                atom_lig_coords = mol.coords[i]
                
                #print(atom_type, mol.symbols[i], len(ind[i]))
                
                if atom_type == "Cb":
                    if mol.symbols[i] == "C":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[7] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[8] += len(ind[i])
                elif atom_type == "Ob":
                    if mol.symbols[i] == "C":
                        desc[9] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[10] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[11] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[12] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[13] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[14] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[15] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[16] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[17] += len(ind[i])
                elif atom_type == "Nb":
                    if mol.symbols[i] == "C":
                        desc[18] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[19] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[20] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[21] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[22] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[23] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[24] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[25] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[26] += len(ind[i])
                elif atom_type == "Cs":
                    if mol.symbols[i] == "C":
                        desc[27] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[28] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[29] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[30] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[31] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[32] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[33] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[34] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[35] += len(ind[i])
                elif atom_type == "Os":
                    if mol.symbols[i] == "C":
                        desc[36] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[37] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[38] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[39] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[40] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[41] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[42] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[43] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[44] += len(ind[i])
                elif atom_type == "Ns":
                    if mol.symbols[i] == "C":
                        desc[45] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[46] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[47] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[48] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[49] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[50] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[51] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[52] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[53] += len(ind[i])
                elif atom_type == "Ss":
                    if mol.symbols[i] == "C":
                        desc[54] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[55] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[56] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[57] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[58] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[59] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[60] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[61] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[62] += len(ind[i])
    return desc

def compute(pdb_id):
    desc_arr = []
    mol_names_arr = []
    for i in range(1, 100):
        try:
            mol_names_arr.append(pdb_id + "_lig_{}".format(i))
            #print(pdb_id, "molnamesarr", mol_names_arr[0])
            mol = read_xyz("./all/" + pdb_id + "/" + pdb_id + "_lig_{}.xyz".format(i))
            pdb = read_pdb("./all/" + pdb_id + "/" + pdb_id + "_prot_1.pdb")

            desc = BB_desc(mol, pdb)
            desc_arr.append(desc)
            
        except Exception as e:
            print("COMPUTE FINISHED")
            break
    return desc_arr, mol_names_arr


def main(): 
    data = pd.read_csv("./index_all.csv")
    codes = [code for code in list(Counter(data.docked_pdb).keys()) if len(code) == 4]
    X = []
    y = []

    try:
        for code in codes:
            desc_arr, mol_names_arr = compute(code)
            print("code, len", code, len(desc_arr), '\n')
            if desc_arr is not None:
                for desc_idx in range(len(desc_arr)):
                    desc_i = desc_arr[desc_idx]
                    mol_name_cur = mol_names_arr[desc_idx]
                    #print("DF SIZE:", mol_name_cur, len(data[data.pose_filename == mol_name_cur]))
                    X.append(desc_i)
                    y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                    #print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
            else:
                X.append(desc_i)
                y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                #print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
    except Exception as e:
        print("error in ", code, mol_name_cur,e)

    X = np.array(X).astype(int)
    y = np.array(y)

    np.save("bb_X_all.npy", X)
    np.save("bb_y_all.npy", y)
        

if __name__ == "__main__":
    main()

COMPUTE FINISHED
code, len 1xal 3 

COMPUTE FINISHED
code, len 2yme 1 

COMPUTE FINISHED
code, len 3uzc 10 

COMPUTE FINISHED
code, len 4ug2 1 

COMPUTE FINISHED
code, len 2ydv 5 

COMPUTE FINISHED
code, len 3qak 1 

COMPUTE FINISHED
code, len 3iar 1 

COMPUTE FINISHED
code, len 5iz6 1 

COMPUTE FINISHED
code, len 5z8h 1 

COMPUTE FINISHED
code, len 1kvl 3 

COMPUTE FINISHED
code, len 4wbg 1 

COMPUTE FINISHED
code, len 2pu4 2 

COMPUTE FINISHED
code, len 4ksq 1 

COMPUTE FINISHED
code, len 4ksp 1 

COMPUTE FINISHED
code, len 6gpx 2 

COMPUTE FINISHED
code, len 4ek4 1 

COMPUTE FINISHED
code, len 4ek6 1 

COMPUTE FINISHED
code, len 4fkj 1 

COMPUTE FINISHED
code, len 4ek8 5 

COMPUTE FINISHED
code, len 3sw4 1 

COMPUTE FINISHED
code, len 3sw7 1 

COMPUTE FINISHED
code, len 4ek5 2 

COMPUTE FINISHED
code, len 4fko 1 

COMPUTE FINISHED
code, len 4fkp 1 

COMPUTE FINISHED
code, len 4fkq 1 

COMPUTE FINISHED
code, len 4fkr 2 

COMPUTE FINISHED
code, len 4fks 1 

COMPUTE FINISHED
code, len 

In [6]:
#BBC12A

import numpy as np
import pandas as pd
from collections import namedtuple
from math import cos, pi, log10
from copy import copy, deepcopy
from sklearn.neighbors import KDTree
import os
from multiprocessing import Pool


Mol = namedtuple('Mol', ['symbols', 'coords', 'message'])

#symbols is a list of atomic name to be met in the pdb
#kdtrees - is a dictionary of the KD-trees for each element

PDB = namedtuple('PDB', ['symbols', 'kdtrees', 'coords'])


def read_xyz(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    message = data[1].rstrip("\n")
    symbols = []
    coords = np.array([]).reshape((0, 3)).astype(float)
    for item in data[2:]:
        if item[0] != "H":
            item = item.strip("\n")
            item = item.split(" ")
            item = list(filter(None, item))
            symbols.append(item[0])
            coords = np.concatenate((coords, np.array(item[1:]).reshape((1,3)).astype(float)), 0)
    mol = Mol(symbols = symbols, coords = coords, message = message)
    mol.coords.astype(float)
    return mol


def read_pdb(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    # retain only lines containing atomic coordinates
    data = [line.rstrip("\n") for line in data if line.startswith("ATOM")]    
    symbols = ['Cb', 'Ob', 'Nb', 'Cs', 'Os', 'Ns', 'Ss']
    coords = {'Cb': [], 'Ob': [], 'Nb': [], 'Cs': [], 'Os': [], 'Ns': [], 'Ss': []}
    # read and keep coordinate per atom type
    for line in data:
        x = float(line[30:37])
        y = float(line[38:45])
        z = float(line[46:53])

        symbol = deepcopy(line[12:16]).strip()          
    
        if symbol in ['C', 'CA', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']:
            coords['Cb'].append([x, y, z])
       
        elif symbol in ['O', 'OA', 'O1', 'O2', 'O3', 'O4', 'O5', 'O6', 'O7', 'O8', 'O9', 'OXT']:
            coords['Ob'].append([x, y, z])
        
        elif symbol in ['N', 'NA', 'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9']:
            coords['Nb'].append([x, y, z])
         
        elif symbol in ['CB','CB1','CB2','CB3','CB4','CB5','CB6','CB7','CB8','CB9','CG','CG1','CG2','CG3','CG4','CG5','CG6','CG7','CG8','CG9','CD','CD1','CD2','CD3','CD4','CD5','CD6','CD7','CD8','CD9','CE','CE1','CE2','CE3','CE4','CE5','CE6','CE7','CE8','CE9','CZ','CZ1','CZ2','CZ3','CZ4','CZ5','CZ6','CZ7','CZ8','CZ9','CH','CH1','CH2','CH3','CH4','CH5','CH6','CH7','CH8','CH9']:
            coords['Cs'].append([x, y, z])
        
        elif symbol in ['OB','OB1','OB2','OB3','OB4','OB5','OB6','OB7','OB8','OB9','OG','OG1','OG2','OG3','OG4','OG5','OG6','OG7','OG8','OG9','OD','OD1','OD2','OD3','OD4','OD5','OD6','OD7','OD8','OD9','OE','OE1','OE2','OE3','OE4','OE5','OE6','OE7','OE8','OE9','OZ','OZ1','OZ2','OZ3','OZ4','OZ5','OZ6','OZ7','OZ8','OZ9','OH','OH1','OH2','OH3','OH4','OH5','OH6','OH7','OH8','OH9']:
            coords['Os'].append([x, y, z])
        
        elif symbol in ['NB','NB1','NB2','NB3','NB4','NB5','NB6','NB7','NB8','NB9','NG','NG1','NG2','NG3','NG4','NG5','NG6','NG7','NG8','NG9','ND','ND1','ND2','ND3','ND4','ND5','ND6','ND7','ND8','ND9','NE','NE1','NE2','NE3','NE4','NE5','NE6','NE7','NE8','NE9','NZ','NZ1','NZ2','NZ3','NZ4','NZ5','NZ6','NZ7','NZ8','NZ9','NH','NH1','NH2','NH3','NH4','NH5','NH6','NH7','NH8','NH9']:
            coords['Ns'].append([x, y, z])
        
        elif symbol in ['SB','SB1','SB2','SB3','SB4','SB5','SB6','SB7','SB8','SB9','SG','SG1','SG2','SG3','SG4','SG5','SG6','SG7','SG8','SG9','SD','SD1','SD2','SD3','SD4','SD5','SD6','SD7','SD8','SD9','SE','SE1','SE2','SE3','SE4','SE5','SE6','SE7','SE8','SE9','SZ','SZ1','SZ2','SZ3','SZ4','SZ5','SZ6','SZ7','SZ8','SZ9','SH','SH1','SH2','SH3','SH4','SH5','SH6','SH7','SH8','SH9', 'S']:
            coords['Ss'].append([x, y, z])
        
        elif symbol == 'SE':
            coords['Ss'].append([x, y, z])
        
        elif symbol in ['LI', 'NA', 'K', 'RB', 'CS', 'AU', 'BA', 'CA', 'CD', 'SR', 'BE', 'CO', 'MG', 'CU', 'FE', 'HG', 'MN', 'NI', 'ZN']:
            continue
        
        elif symbol.strip()[0] == 'H':
            continue
        
        else:
            print(symbol, line[:])
            raise ValueError("Unknown atomic symbol in pdb")

    kdtrees = {}

    # create a dict of KD trees
    for atom_type in symbols:
        coords[atom_type] = np.array(coords[atom_type])
        if len(coords[atom_type]) != 0:
            kdtrees[atom_type] = KDTree(coords[atom_type], leaf_size=10)

    mol = PDB(symbols=symbols, kdtrees=kdtrees, coords=coords)

    return mol

 
     

def BB_desc(mol, pdb):

    desc = [0.] * 7
    for atom_type in pdb.coords.keys():
        # check if the atom is in the structure
        # if not add zeros for the vector
        if len(pdb.coords[atom_type]) != 0:
            ind = pdb.kdtrees[atom_type].query_radius(mol.coords, r = 12)

            atom_type_list = []
            for i in range(mol.coords.shape[0]):
                
                atom_env_coords = pdb.coords[atom_type][ind[i], :]
                atom_lig_coords = mol.coords[i]
                
                #print(atom_type, mol.symbols[i], len(ind[i]))
                
                if atom_type == "Cb":
                    if mol.symbols[i] == "C":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[0] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[0] += len(ind[i])
                elif atom_type == "Ob":
                    if mol.symbols[i] == "C":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[1] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[1] += len(ind[i])
                elif atom_type == "Nb":
                    if mol.symbols[i] == "C":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[2] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[2] += len(ind[i])
                elif atom_type == "Cs":
                    if mol.symbols[i] == "C":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[3] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[3] += len(ind[i])
                elif atom_type == "Os":
                    if mol.symbols[i] == "C":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[4] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[4] += len(ind[i])
                elif atom_type == "Ns":
                    if mol.symbols[i] == "C":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[5] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[5] += len(ind[i])
                elif atom_type == "Ss":
                    if mol.symbols[i] == "C":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "O":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "N":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "S":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "P":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "F":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "Cl":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "Br":
                        desc[6] += len(ind[i])
                    elif mol.symbols[i] == "I":
                        desc[6] += len(ind[i])
    return desc

def compute(pdb_id):
    desc_arr = []
    mol_names_arr = []
    for i in range(1, 100):
        try:
            mol_names_arr.append(pdb_id + "_lig_{}".format(i))
            #print(pdb_id, "molnamesarr", mol_names_arr[0])
            mol = read_xyz("./all/" + pdb_id + "/" + pdb_id + "_lig_{}.xyz".format(i))
            pdb = read_pdb("./all/" + pdb_id + "/" + pdb_id + "_prot_1.pdb")

            desc = BB_desc(mol, pdb)
            desc_arr.append(desc)
            
        except Exception as e:
            print("COMPUTE FINISHED")
            break
    return desc_arr, mol_names_arr


def main(): 
    data = pd.read_csv("./index_all.csv")
    codes = [code for code in list(Counter(data.docked_pdb).keys()) if len(code) == 4]
    X = []
    y = []

    try:
        for code in codes:
            desc_arr, mol_names_arr = compute(code)
            print("code, len", code, len(desc_arr), '\n')
            if desc_arr is not None:
                for desc_idx in range(len(desc_arr)):
                    desc_i = desc_arr[desc_idx]
                    mol_name_cur = mol_names_arr[desc_idx]
                    #print("DF SIZE:", mol_name_cur, len(data[data.pose_filename == mol_name_cur]))
                    X.append(desc_i)
                    y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                    #print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
            else:
                X.append(desc_i)
                y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))
                #print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
    except Exception as e:
        print("error in ", code, mol_name_cur,e)

    X = np.array(X).astype(int)
    y = np.array(y)

    np.save("bbc_X_all.npy", X)
    np.save("bbc_y_all.npy", y)
        

if __name__ == "__main__":
    main()

COMPUTE FINISHED
code, len 1xal 3 

COMPUTE FINISHED
code, len 2yme 1 

COMPUTE FINISHED
code, len 3uzc 10 

COMPUTE FINISHED
code, len 4ug2 1 

COMPUTE FINISHED
code, len 2ydv 5 

COMPUTE FINISHED
code, len 3qak 1 

COMPUTE FINISHED
code, len 3iar 1 

COMPUTE FINISHED
code, len 5iz6 1 

COMPUTE FINISHED
code, len 5z8h 1 

COMPUTE FINISHED
code, len 1kvl 3 

COMPUTE FINISHED
code, len 4wbg 1 

COMPUTE FINISHED
code, len 2pu4 2 

COMPUTE FINISHED
code, len 4ksq 1 

COMPUTE FINISHED
code, len 4ksp 1 

COMPUTE FINISHED
code, len 6gpx 2 

COMPUTE FINISHED
code, len 4ek4 1 

COMPUTE FINISHED
code, len 4ek6 1 

COMPUTE FINISHED
code, len 4fkj 1 

COMPUTE FINISHED
code, len 4ek8 5 

COMPUTE FINISHED
code, len 3sw4 1 

COMPUTE FINISHED
code, len 3sw7 1 

COMPUTE FINISHED
code, len 4ek5 2 

COMPUTE FINISHED
code, len 4fko 1 

COMPUTE FINISHED
code, len 4fkp 1 

COMPUTE FINISHED
code, len 4fkq 1 

COMPUTE FINISHED
code, len 4fkr 2 

COMPUTE FINISHED
code, len 4fks 1 

COMPUTE FINISHED
code, len 

In [7]:
#SS312A

import numpy as np
import pandas as pd
from collections import namedtuple
from math import log10
from copy import copy, deepcopy
from sklearn.neighbors import KDTree
from collections import Counter
import os
import sys
np.set_printoptions(threshold=sys.maxsize)
from Bio import SeqUtils


Mol = namedtuple('Mol', ['symbols', 'coords', 'message'])

#symbols is a list of atomic name to be met in the pdb
#kdtrees - is a dictionary of the KD-trees for each atom

PDB = namedtuple('PDB', ['symbols', 'kdtrees', 'coords'])

#res_types is a dictionary of residue information (name, chain, number) and its secondary structure info

Sec = namedtuple('Sec', ['res_types'])


def read_xyz(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    message = data[1].rstrip("\n")
    symbols = []
    coords = np.array([]).reshape((0, 3)).astype(float)
    for item in data[2:]:
        if item[0] != "H":
            item = item.strip("\n")
            item = item.split(" ")
            item = list(filter(None, item))
            symbols.append(item[0])
            coords = np.concatenate((coords, np.array(item[1:]).reshape((1,3)).astype(float)), 0)
    mol = Mol(symbols = symbols, coords = coords, message = message)
    mol.coords.astype(float)
    return mol


def read_pdb(filename):
    f = open(filename, 'r')
    data = f.readlines()
    f.close()
    #retain only lines containing atomic coordinates
    data = [line.rstrip("\n") for line in data if line.startswith("ATOM") or line.startswith("HETATM")]


    xyz = []
    symbols = []

    for lines in data:  
        if (lines[11:17].strip(' ')[0]) != ('H') and (lines[11:17].strip(' ')[0]) not in ('K'):
            if ((((lines[71:80])).strip(' ')[:2]).upper()) not in  ('LI', 'NA', 'RB', 'CS', 
                                                     'AU', 'BA', 'CA', 'CD', 'SR', 
                                                     'BE', 'CO', 'MG', 'CU', 'FE', 
                                                     'HG', 'MN', 'NI', 'ZN'):
                    symbols.append((lines[16:20].strip(' '), lines[21:22], lines[23:26].strip(' ')))
                    x = float(lines[30:37])
                    y = float(lines[38:45])
                    z = float(lines[46:53])
                    xyz.append([x, y, z])

    res_coord_pair = []

    for i in range(len(symbols)):
        res_coord_pair.append((symbols[i], xyz[i]))


    unique_resnames = (list(Counter(symbols).keys()))

    coords = {}

    for res in res_coord_pair:
        res_cur = res[0]
        coord_cur = res[1]
        if res_cur not in coords.keys():
            coords[res_cur] = []
        coords[res_cur].append(coord_cur)

    kdtrees = {}

    #create a dict of KD trees
    for res in coords.keys():
        #coords[res] = np.array(coords[res])
        kdtrees[res] = KDTree(coords[res], leaf_size=10)

    pdb = PDB(symbols=unique_resnames, kdtrees=kdtrees, coords=coords)
    return pdb

 
def read_dssp(filename):
    f = open(filename, 'r')
    readl = f.readlines()
    f.close()
    dssps = [line.rstrip("\n") for line in readl[28:]]    

    ssr = []
    for lines in dssps:
            ssr.append((lines[13:14], lines[11:13].strip(' '), lines[7:11].strip(' '), lines[16:17]))

    ss=[]
    for lines in ssr:
        if lines != ('!', '', '', ' '):
            
            ss.append(lines)

    residues=[]
    
    resnames=[]
    reschains=[]
    resnums=[]

    for n in range(len(ss)):
        resnames.append((str(SeqUtils.IUPACData.protein_letters_1to3.get(ss[n][0]))).upper())
        reschains.append(ss[n][1])
        resnums.append(ss[n][2])
        residues.append((resnames[n], reschains[n], resnums[n]))
    
    restypes=[]
    
    for n in range(len(ss)):
        for i in (ss[n][3]):
            if i == (' '):
                i = 'R'
            restypes.append(i)
            
    # restypes is a dict of residues vs types
    res_types = {}

    res_types = dict(zip(residues, restypes))

    sec = Sec(res_types = res_types)
    return sec     


def SS_desc(mol, pdb, sec):
    desc = [0.] * 3
    for atom_type in pdb.coords.keys():
        # check if the residue is in the structure
        # if not add zeros for the vector
            ind = pdb.kdtrees[atom_type].query_radius(mol.coords, r = 12)
            counts = ind
            for i in range(len(ind)):
                if len(counts[i]) != 0:
                    counts[i] = np.array(1)
                else:
                    counts[i] = np.array(0)
            #for i in range(mol.coords.shape[0]):

                #print(atom_type, mol.symbols[i], len(ind[i]))

                if sec.res_types.get(atom_type) == "H" or sec.res_types.get(atom_type) == "G" or sec.res_types.get(atom_type) == "I":
                    if mol.symbols[i] == "C":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[0] = desc[0] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[0] = desc[0] + counts[i]

                if sec.res_types.get(atom_type) == "B" or sec.res_types.get(atom_type) == "E":
                    if mol.symbols[i] == "C":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[1] = desc[1] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[1] = desc[1] + counts[i]

                if sec.res_types.get(atom_type) == "T" or sec.res_types.get(atom_type) == "S" or sec.res_types.get(atom_type) == "R":
                    if mol.symbols[i] == "C":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "O":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "N":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "S":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "P":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "F":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "Cl":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "Br":
                        desc[2] = desc[2] + counts[i]
                    elif mol.symbols[i] == "I":
                        desc[2] = desc[2] + counts[i]

    return desc

def compute(pdb_id):
    desc_arr = []
    mol_names_arr = []
    for i in range(1, 100):
        try:
            mol_names_arr.append(pdb_id + "_lig_{}".format(i))
            #print(pdb_id, "molnamesarr", mol_names_arr[-1])
            mol = read_xyz("./all/" + pdb_id + "/" + pdb_id + "_lig_{}.xyz".format(i))
            pdb = read_pdb("./all/" + pdb_id + "/" + pdb_id + "_prot_1.pdb")
            sec = read_dssp("./all/" + pdb_id + "/" + pdb_id + "_sec_1.dssp")

            desc = SS_desc(mol, pdb, sec)
            desc_arr.append(desc)

        except Exception as e:
            print("COMPUTE FINISHED", e)
            break
    return desc_arr, mol_names_arr


def main(): 
    data = pd.read_csv("./index_all.csv")
    codes = [code for code in list(Counter(data.docked_pdb).keys()) if len(code) == 4]
    X = []
    y = []
    try:
        for code in codes:
            desc_arr, mol_names_arr = compute(code)
            print("code, len", code, len(desc_arr), '\n')
            if desc_arr is not None:
                for desc_idx in range(len(desc_arr)):
                    desc_i = desc_arr[desc_idx]
                    mol_name_cur = mol_names_arr[desc_idx]
                    print("DF SIZE:", mol_name_cur, len(data[data.pose_filename == mol_name_cur]))
                    X.append(desc_i)
                    y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))

                    #print(desc_i, data[data.pose_filename == mol_name_cur].koff_s.iloc[0])
            else:
                X.append(desc_i)
                y.append(-log10(data[data.pose_filename == mol_name_cur].koff_s.iloc[0]))

    except Exception as e:
        print("error in ", code, mol_name_cur,e)

    X = np.array(X).astype(int)
    y = np.array(y).astype(float)

    np.save("ss3_X_all.npy", X)
    np.save("ss3_y_all.npy", y)    

if __name__ == "__main__":
    main()

COMPUTE FINISHED [Errno 2] No such file or directory: './all/1xal/1xal_lig_4.xyz'
code, len 1xal 3 

DF SIZE: 1xal_lig_1 1
DF SIZE: 1xal_lig_2 1
DF SIZE: 1xal_lig_3 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/2yme/2yme_lig_2.xyz'
code, len 2yme 1 

DF SIZE: 2yme_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3uzc/3uzc_lig_11.xyz'
code, len 3uzc 10 

DF SIZE: 3uzc_lig_1 1
DF SIZE: 3uzc_lig_2 1
DF SIZE: 3uzc_lig_3 1
DF SIZE: 3uzc_lig_4 1
DF SIZE: 3uzc_lig_5 1
DF SIZE: 3uzc_lig_6 1
DF SIZE: 3uzc_lig_7 1
DF SIZE: 3uzc_lig_8 1
DF SIZE: 3uzc_lig_9 1
DF SIZE: 3uzc_lig_10 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/4ug2/4ug2_lig_2.xyz'
code, len 4ug2 1 

DF SIZE: 4ug2_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/2ydv/2ydv_lig_6.xyz'
code, len 2ydv 5 

DF SIZE: 2ydv_lig_1 1
DF SIZE: 2ydv_lig_2 1
DF SIZE: 2ydv_lig_3 1
DF SIZE: 2ydv_lig_4 1
DF SIZE: 2ydv_lig_5 1
COMPUTE FINISHED [Errno 2] No such file or directory

COMPUTE FINISHED [Errno 2] No such file or directory: './all/2aay/2aay_lig_3.xyz'
code, len 2aay 2 

DF SIZE: 2aay_lig_1 1
DF SIZE: 2aay_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3nrc/3nrc_lig_6.xyz'
code, len 3nrc 5 

DF SIZE: 3nrc_lig_1 1
DF SIZE: 3nrc_lig_2 1
DF SIZE: 3nrc_lig_3 1
DF SIZE: 3nrc_lig_4 1
DF SIZE: 3nrc_lig_5 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/4no7/4no7_lig_3.xyz'
code, len 4no7 2 

DF SIZE: 4no7_lig_1 1
DF SIZE: 4no7_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3f9m/3f9m_lig_3.xyz'
code, len 3f9m 2 

DF SIZE: 3f9m_lig_1 1
DF SIZE: 3f9m_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/1jyq/1jyq_lig_3.xyz'
code, len 1jyq 2 

DF SIZE: 1jyq_lig_1 1
DF SIZE: 1jyq_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ldo/3ldo_lig_2.xyz'
code, len 3ldo 1 

DF SIZE: 3ldo_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ldp/3ldp_lig_8.xyz'
code

COMPUTE FINISHED [Errno 2] No such file or directory: './all/6f1n/6f1n_lig_2.xyz'
code, len 6f1n 1 

DF SIZE: 6f1n_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/6elo/6elo_lig_3.xyz'
code, len 6elo 2 

DF SIZE: 6elo_lig_1 1
DF SIZE: 6elo_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/5j64/5j64_lig_5.xyz'
code, len 5j64 4 

DF SIZE: 5j64_lig_1 1
DF SIZE: 5j64_lig_2 1
DF SIZE: 5j64_lig_3 1
DF SIZE: 5j64_lig_4 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/6eln/6eln_lig_2.xyz'
code, len 6eln 1 

DF SIZE: 6eln_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/5j20/5j20_lig_3.xyz'
code, len 5j20 2 

DF SIZE: 5j20_lig_1 1
DF SIZE: 5j20_lig_2 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/5j86/5j86_lig_9.xyz'
code, len 5j86 8 

DF SIZE: 5j86_lig_1 1
DF SIZE: 5j86_lig_2 1
DF SIZE: 5j86_lig_3 1
DF SIZE: 5j86_lig_4 1
DF SIZE: 5j86_lig_5 1
DF SIZE: 5j86_lig_6 1
DF SIZE: 5j86_lig_7 1
DF SIZE: 5j86_lig_8 

COMPUTE FINISHED [Errno 2] No such file or directory: './all/5if1/5if1_lig_4.xyz'
code, len 5if1 3 

DF SIZE: 5if1_lig_1 1
DF SIZE: 5if1_lig_2 1
DF SIZE: 5if1_lig_3 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/1w98/1w98_lig_4.xyz'
code, len 1w98 3 

DF SIZE: 1w98_lig_1 1
DF SIZE: 1w98_lig_2 1
DF SIZE: 1w98_lig_3 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/3ddu/3ddu_lig_4.xyz'
code, len 3ddu 3 

DF SIZE: 3ddu_lig_1 1
DF SIZE: 3ddu_lig_2 1
DF SIZE: 3ddu_lig_3 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/1bil/1bil_lig_8.xyz'
code, len 1bil 7 

DF SIZE: 1bil_lig_1 1
DF SIZE: 1bil_lig_2 1
DF SIZE: 1bil_lig_3 1
DF SIZE: 1bil_lig_4 1
DF SIZE: 1bil_lig_5 1
DF SIZE: 1bil_lig_6 1
DF SIZE: 1bil_lig_7 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/2v0z/2v0z_lig_2.xyz'
code, len 2v0z 1 

DF SIZE: 2v0z_lig_1 1
COMPUTE FINISHED [Errno 2] No such file or directory: './all/6c4d/6c4d_lig_34.xyz'
code, len 6c4d 33 

DF SIZE: 6c4d_lig_